In [1]:
'''
Set up your source space
You need a discretization of the cortical sheet (or volume) into a set of dipoles:
'''

import mne

# point MNE at your FreeSurfer SUBJECTS_DIR
subjects_dir = '/path/to/freesurfer/subjects'

# create a source space on the cortical surface (e.g. ~10 000 vertices / hemisphere)
src = mne.setup_source_space(
    subject='subj01',
    spacing='oct6',                # roughly 4098 sources per hemi; try 'oct4' for fewer
    subjects_dir=subjects_dir,
    add_dist=False                 # save memory if you won’t do source connectivity
)

FileNotFoundError: subjects_dir does not exist: "/path/to/freesurfer/subjects"

In [ ]:
'''
Build a BEM model & solution
This models how currents flow through head tissues:
'''

# 2a. Create a 3-layer BEM model from the MRI (watershed surfaces)
model = mne.make_bem_model(
    subject='subj01',
    ico=4,                         # mesh granularity: higher ico → finer mesh
    conductivity=(0.3, 0.006, 0.3),  # scalp, skull, brain in S/m
    subjects_dir=subjects_dir
)

# 2b. Compute the BEM solution
bem = mne.make_bem_solution(model)

In [ ]:
'''
Compute the forward solution
You need your EEG sensor locations coregistered to the head:
'''

# 'trans' is your head<->MRI alignment file (e.g. from mne.gui.coregistration)
trans = 'subj01-trans.fif'

# 'info' comes from your Raw object: raw.info
fwd = mne.make_forward_solution(
    info=raw.info,
    trans=trans,
    src=src,
    bem=bem,
    meg=False,       # if you only have EEG
    eeg=True,
    mindist=5.0,     # mm minimum distance of sources from inner skull
    n_jobs=4
)

In [ ]:
'''
Estimate the noise covariance
This quantifies the sensor space noise you will regularize against:
'''

# If you have empty‐room data:
# noise_cov = mne.compute_raw_covariance(raw_empty, tmin=0, tmax=None)

# Or use pre‐stimulus baseline from your epochs:
epochs = mne.read_epochs('subj01-epo.fif')
noise_cov = mne.compute_covariance(
    epochs, 
    tmin=-0.2,      # baseline window start (s)
    tmax=0.0,       # baseline window end (s)
    method='empirical'
)

In [ ]:
'''
Make the inverse operator
Now combine info, fwd, and noise_cov:
'''

from mne.minimum_norm import make_inverse_operator

inverse_operator = make_inverse_operator(
    info=raw.info,
    forward=fwd,
    noise_cov=noise_cov,
    loose=0.2,      # for cortical orientation constraint (0 = fixed, 1 = free)
    depth=0.8,      # depth weighting exponent (0–1)
    fixed=False     # if True, solves only for the normal orientation
)

'''
    - loose controls how “tangential” vs. “normal” dipoles are penalized.
    - depth balances bias toward superficial sources.
    - fixed toggles a fixed‐orientation vs. full 3D dipole estimate.
'''

In [ ]:
'''
You can save it for later:
'''

mne.minimum_norm.write_inverse_operator(
    'subj01-inv.fif', 
    inverse_operator
)